Inference\
https://wikidocs.net/115055

https://huggingface.co/docs/transformers/index

http://kkma.snu.ac.kr/documents/index.jsp?doc=postag


# Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
# Date; 2. 10. 2022 - 2. 22. 2022
# Title: Korean_NER
# Professor: Seung-Hoon Na

In [24]:
import os
import pandas as pd
import json
import transformers
from transformers import BertTokenizer
# from tqdm.auto import tqdm

In [25]:
PATH_dir = 'C:\\Users\\LG\\Desktop\\github\\JBNU-2022-SPRING\\English world class tagging & Korean_Named Entity Recognition\\Ko_En_NER_POStag_data\Ko_NER_POS'
file_name_ko_train = 'train.txt'
file_name_ko_test = 'test.txt'
file_name_ko_dev = 'dev.txt'

In [38]:
file_name = file_name_ko_train
PATH_data = os.path.join(PATH_dir, file_name)

# change file extention from '.txt' to '.json'
prepro_file_name = 'prepro_' + file_name.rsplit('.')[0] + '.json'
PATH_preprodata = os.path.join(PATH_dir, prepro_file_name)

In [39]:
with open(PATH_data, 'r', encoding='UTF8') as f:
    rawdata = f.readlines()

In [40]:
df0 = pd.DataFrame(rawdata)
df0

,0
0,특히/MAG\tMAG\tB\tO\n
1,김병현/NNP\tNNP\tB\tB-PS\n
2,은/JX\tJX\tI\tO\n
3,4/SN\tSN\tB\tO\n
4,회/NNB\tNNB\tI\tO\n
...,...
133782,고/EC\tEC\tI\tO\n
133783,있/VX\tVX\tB\tO\n
133784,다/EF\tEF\tI\tO\n
133785,./SF\tSF\tB\tO\n


In [41]:
df = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2','1-3'])
df

,1-1,1-2,1-3
0,특히,MAG\tMAG\tB\tO\n,None
1,김병현,NNP\tNNP\tB\tB-PS\n,None
2,은,JX\tJX\tI\tO\n,None
3,4,SN\tSN\tB\tO\n,None
4,회,NNB\tNNB\tI\tO\n,None
...,...,...,...
133782,고,EC\tEC\tI\tO\n,None
133783,있,VX\tVX\tB\tO\n,None
133784,다,EF\tEF\tI\tO\n,None
133785,.,SF\tSF\tB\tO\n,None


In [42]:
df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
df2

,1,2,3,4
0,특히/MAG,MAG,B,O\n
1,김병현/NNP,NNP,B,B-PS\n
2,은/JX,JX,I,O\n
3,4/SN,SN,B,O\n
4,회/NNB,NNB,I,O\n
...,...,...,...,...
133782,고/EC,EC,I,O\n
133783,있/VX,VX,B,O\n
133784,다/EF,EF,I,O\n
133785,./SF,SF,B,O\n


In [50]:
df7 = pd.concat([df['1-1'],df2['4'].str.strip('\n')],axis = 1)
df7 = pd.DataFrame(df7)
df7 = df7.rename(columns={'1-1':'Entity','4':'tag'})
df7[:50]

,Entity,tag
0,특히,O
1,김병현,B-PS
2,은,O
3,4,O
4,회,O
5,말,O
6,에,O
7,무,O
8,기력,O
9,하,O


In [54]:
print(len(df7))

133787


In [55]:
df7['Entity'][3]

'4'

# Preprocessing 1

만약, BERT의 단어 집합에 em, ##bed, ##ding, #s라는 서브 워드들이 존재한다면, embeddings는 em, ##bed, ##ding, #s로 분리됩니다. 여기서 ##은 이 서브워드들은 단어의 중간부터 등장하는 서브워드라는 것을 알려주기 위해 단어 집합 생성 시 표시해둔 기호입니다. 이런 표시가 있어야만 em, ##bed, ##ding, #s를 다시 손쉽게 embeddings로 복원할 수 있을 것

In [56]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)

In [77]:
lst=[]
lst_set=[]
lst_ent=[]
lst_tag=[]
for i in range(len(df7)):
    if df7['tag'][i] != None:
        lst_ent.append(df7['Entity'][i])
        lst_tag.append(df7['tag'][i])
    else:
        lst_set.append(lst_ent)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        lst_ent = []
        lst_tag = []
        lst_set = [] 

In [79]:
print('entity', len(lst[3][0]))
print(lst[3][0])

print('token', len(lst[3][1]))
print(lst[3][1])

entity 41
['우리', '금융', '그룹', '은', '19', '일', '지난주', '부터', '네덜란드', '프로', '축구', '에인트호벤', '의', '미드필더', '박지성', '과', '미국', '프로', '야구', '텍사스', '레인저스', '의', '투수', '박찬호', '를', '모델', '로', '기용', ',', '공중파', '텔레비전', '을', '통해', '광고', '를', '재개', '했', '다고', '밝혔', '다', '.']
token 41
['B-OG', 'I-OG', 'I-OG', 'O', 'B-DT', 'I-DT', 'B-DT', 'I-DT', 'B-LC', 'O', 'O', 'B-OG', 'O', 'O', 'B-PS', 'O', 'B-LC', 'O', 'O', 'B-OG', 'I-OG', 'O', 'O', 'B-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [80]:
lst=[]
lst_set = []
lst_ent = []
lst_tag = []
for i in range(len(df7)):
    if df7['tag'][i] != None:
        lst_ent.append(df7['Entity'][i])
        lst_tag.append(df7['tag'][i])

    else:
        lst_set.append(lst_ent)
        result = ' '.join(str(s) for s in lst_ent)
        morph_to_tokens = tokenizer.tokenize(result)
        lst_set.append(morph_to_tokens)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        result = 0
        lst_ent = []
        lst_tok = []
        lst_tag = []
        lst_set = [] 

In [81]:
print('entity', len(lst[3][0]))
print(lst[3][0])

print('token', len(lst[3][1]))
print(lst[3][1])

print('entity_tag', len(lst[3][2]))
print(lst[3][2])

entity 41
['우리', '금융', '그룹', '은', '19', '일', '지난주', '부터', '네덜란드', '프로', '축구', '에인트호벤', '의', '미드필더', '박지성', '과', '미국', '프로', '야구', '텍사스', '레인저스', '의', '투수', '박찬호', '를', '모델', '로', '기용', ',', '공중파', '텔레비전', '을', '통해', '광고', '를', '재개', '했', '다고', '밝혔', '다', '.']
token 74
['우', '##리', '금', '##융', '그', '##룹', '은', '19', '일', '지', '##난', '##주', '부', '##터', '네', '##덜', '##란드', '프로', '축구', '에', '##인', '##트', '##호', '##벤', '의', '미', '##드', '##필', '##더', '박', '##지', '##성', '과', '미국', '프로', '야구', '텍', '##사', '##스', '레', '##인', '##저', '##스', '의', '투', '##수', '박', '##찬', '##호', '를', '모', '##델', '로', '기', '##용', ',', '공', '##중', '##파', '텔레비전', '을', '통해', '광', '##고', '를', '재', '##개', '했', '다', '##고', '밝', '##혔', '다', '.']
entity_tag 41
['B-OG', 'I-OG', 'I-OG', 'O', 'B-DT', 'I-DT', 'B-DT', 'I-DT', 'B-LC', 'O', 'O', 'B-OG', 'O', 'O', 'B-PS', 'O', 'B-LC', 'O', 'O', 'B-OG', 'I-OG', 'O', 'O', 'B-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [85]:
morph_to_tokens = tokenizer.tokenize(lst[3][0])
print(morph_to_tokens)

['우', '##리', '금', '##융', '그', '##룹', '은', '19', '일', '지', '##난', '##주', '부', '##터', '네', '##덜', '##란드', '프로', '축구', '에', '##인', '##트', '##호', '##벤', '의', '미', '##드', '##필', '##더', '박', '##지', '##성', '과', '미국', '프로', '야구', '텍', '##사', '##스', '레', '##인', '##저', '##스', '의', '투', '##수', '박', '##찬', '##호', '를', '모', '##델', '로', '기', '##용', ',', '공', '##중', '##파', '텔레비전', '을', '통해', '광', '##고', '를', '재', '##개', '했', '다', '##고', '밝', '##혔', '다', '.']


# Tokenizer tagging

In [88]:
lst=[]
lst_set = []
lst_ent = []
lst_tag = []
for i in range(len(df7)):
    if df7['tag'][i] != None:
        morph_to_tokens = tokenizer.tokenize(df7['Entity'][i])
        if 1 == len(morph_to_tokens):
            lst_ent.append(df7['Entity'][i])
            lst_tag.append(df7['tag'][i])
            
        else:
            if df7['tag'][i] == 'O':
                lst_ent.append(df7['Entity'][i])
                lst_tag.append(df7['tag'][i])
                for i in range(len(morph_to_tokens)-1):
                    lst_tag.append(df7['tag'][i])
                
            else:
                lst_ent.append(df7['Entity'][i])
                lst_tag.append(df7['tag'][i])
                split = df7['tag'][i].split('-')
                for i in range(len(morph_to_tokens)-1):
                    results = 'I-' + split[-1]
                    lst_tag.append(results)
            
    else:
        lst_ent = result = ' '.join(str(s) for s in lst_ent)
        lst_set.append(lst_ent)
        morph_to_tokens = tokenizer.tokenize(lst_ent)
        lst_set.append(morph_to_tokens)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        result = 0
        lst_ent = []
        lst_tok = []
        lst_tag = []
        lst_set = [] 

In [89]:
print('entity', len(lst[3][0]))
print(lst[3][0])

print('token', len(lst[3][1]))
print(lst[3][1])

print('token_tag', len(lst[3][2]))
print(lst[3][2])

entity 125
우리 금융 그룹 은 19 일 지난주 부터 네덜란드 프로 축구 에인트호벤 의 미드필더 박지성 과 미국 프로 야구 텍사스 레인저스 의 투수 박찬호 를 모델 로 기용 , 공중파 텔레비전 을 통해 광고 를 재개 했 다고 밝혔 다 .
token 74
['우', '##리', '금', '##융', '그', '##룹', '은', '19', '일', '지', '##난', '##주', '부', '##터', '네', '##덜', '##란드', '프로', '축구', '에', '##인', '##트', '##호', '##벤', '의', '미', '##드', '##필', '##더', '박', '##지', '##성', '과', '미국', '프로', '야구', '텍', '##사', '##스', '레', '##인', '##저', '##스', '의', '투', '##수', '박', '##찬', '##호', '를', '모', '##델', '로', '기', '##용', ',', '공', '##중', '##파', '텔레비전', '을', '통해', '광', '##고', '를', '재', '##개', '했', '다', '##고', '밝', '##혔', '다', '.']
token_tag 74
['B-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'O', 'B-DT', 'I-DT', 'B-DT', 'I-DT', 'I-DT', 'I-DT', 'I-DT', 'B-LC', 'I-LC', 'I-LC', 'O', 'O', 'B-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'O', 'O', 'O', 'B-PS', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'B-LC', 'O', 'O', 'B-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'O', 'O', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [62]:
with open(PATH_preprodata, 'w') as f:
    json.dump(lst, f)

In [63]:
with open(PATH_preprodata, 'r') as f:
    preprodata = json.load(f)

# Example

# My Data

In [90]:
result = ' '.join(str(s) for s in lst[3][0])

In [91]:
inputs = tokenizer(result, return_tensors='pt')
print(inputs['input_ids'].size())
inputs

torch.Size([1, 87])


{'input_ids': tensor([[ 101, 9604, 9238, 8928, 9629, 8924, 9218, 9632,  122,  130, 9641, 9706,
         8984, 9689, 9365, 9861, 9011, 9077, 9159, 9113, 9942, 9202, 9766, 8908,
         9559, 9640, 9890, 9985, 9346, 9637, 9309, 9113, 9949, 9074, 9319, 9706,
         9434, 8898, 9309, 8909, 9942, 9202, 9538, 8908, 9867, 9405, 9477, 9186,
         9640, 9663, 9477, 9637, 9881, 9460, 9319, 9732, 9985, 9233, 9283, 9086,
         9202, 8932, 9603,  117, 8896, 9694, 9901, 9869, 9186, 9379, 9665, 9633,
         9879, 9960, 8903, 8888, 9233, 9659, 8857, 9965, 9056, 8888, 9324, 9982,
         9056,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [92]:
from transformers import BertModel

In [93]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
bert = BertModel.from_pretrained(PRETAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [94]:
outputs = bert(**inputs)

In [95]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [96]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 2.5414e-01, -9.2687e-02, -3.6224e-01,  ..., -1.4949e-02,
           5.6352e-02, -6.9689e-02],
         [ 8.1077e-01, -4.8293e-01,  3.3429e-01,  ..., -1.4545e-01,
           7.5083e-02, -8.7050e-04],
         [ 1.0681e+00, -2.7797e-01, -2.6868e-01,  ..., -1.1319e-01,
           1.3418e-01, -5.6471e-02],
         ...,
         [-5.8204e-05,  2.0799e-01,  8.6430e-02,  ..., -3.0371e-01,
          -1.4953e-01, -2.0159e-01],
         [ 2.7187e-01, -2.0264e-01, -6.3314e-01,  ..., -1.5875e-01,
           9.3204e-03, -9.7499e-02],
         [ 4.2106e-01, -1.4425e-01, -3.5376e-01,  ..., -2.6815e-01,
           1.9158e-01, -1.2440e-01]]], grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 9.0856e-02, -6.3134e-02,  1.7909e-01, -7.9653e-02, -7.7019e-02,
          1.7029e-01,  1.0738e-01,  9.8361e-02, -3.9882e-01,  1.6635e-01,
         -1.1500e-02, -9.2529e-02, -1.2946e-01,  1.8789e-02,  1.5700e-01,
          7.0517e-0